In [50]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [52]:
# Function to extract Product Title   Functions Which i have taken
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [ ]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.com/s?k=playstation+4&ref=nb_sb_noss_2"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [59]:
amazon_df

,title,price,rating,reviews,availability
0,"Playstation SONY 4, 500GB Slim System [CUH-221...",,4.3 out of 5 stars,491 ratings,Not Available
1,PlayStation®4 Console – Call of Duty® Modern W...,,4.4 out of 5 stars,125 ratings,In Stock
2,Sony PlayStation 4 Slim 500GB Premium Bundle (...,,Previous set of slides,,Not Available
3,PlayStation 4 Slim 1TB Console,,4.7 out of 5 stars,"15,778 ratings",In Stock
4,"Wuthur PS4 Controller Wireless 2 Pack, Compati...",,4.3 out of 5 stars,266 ratings,In Stock
5,"FASIGO PS4 Controller 2 Pack, Wireless PS4 Con...",,4.2 out of 5 stars,613 ratings,In Stock
6,Sonic Frontiers - PlayStation 4,,4.7 out of 5 stars,989 ratings,In Stock
7,"PS4 Controller Charger Dock Station, OIVO 1.8H...",,4.7 out of 5 stars,"39,767 ratings",In Stock
8,NBA 2K25 - PlayStation 4,,4.6 out of 5 stars,81 ratings,In Stock
9,$250 PlayStation Store Gift Card [Digital Code],,4.6 out of 5 stars,"263,407 ratings",Available now
